# Setup

## Imports

In [ ]:
import feedparser
import trafilatura
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
import torch
from transformers import pipeline
import spacy
from collections import Counter, defaultdict
import re
from pathlib import Path
import csv
from youtube_transcript_api import YouTubeTranscriptApi
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
# import io
from dotenv import load_dotenv
from googleapiclient.discovery import build
import os
from youtube_transcript_api.proxies import WebshareProxyConfig
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
import json
import time
import random
from pathlib import Path
import requests
from tqdm import tqdm
load_dotenv()

True

## Spacy

In [35]:
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Installing spaCy model...")
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load("en_core_web_sm")


## Sentiment Analyzer

In [36]:
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="ProsusAI/finbert",
    tokenizer="ProsusAI/finbert"
)

def get_text_sentiment_score(text: str, max_chars=512) -> float:
    if not text or not text.strip():
        return 0.0

    chunks = [text[i:i+max_chars] for i in range(0, len(text), max_chars)]
    scores = []

    for chunk in chunks:
        result = sentiment_analyzer(chunk)[0]
        label = result["label"].upper()
        score = float(result["score"])

        if "POS" in label:
            scores.append(score)
        elif "NEG" in label:
            scores.append(-score)
        else:
            scores.append(0.0)

    return sum(scores) / len(scores) if scores else 0.0

Device set to use cpu


## Entity Mentions

In [37]:
FEEDS = [
    "https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=10000664",
]

MAX_ARTICLES_PER_FEED = 30
CONTEXT_SENTENCES = 1
TICKER_LIST_PATH = Path("tickers.csv")  # optional: columns ticker,name

TICKER_RE = re.compile(r"(?<![A-Z])\$?[A-Z]{2,5}(?![A-Z])")
TICKER_STOP = {
    "A", "AN", "AND", "ARE", "AS", "AT", "BE", "BUT", "BY", "CAN", "CO", "FOR",
    "FROM", "HAS", "HAVE", "IN", "IS", "IT", "ITS", "NOT", "OF", "ON", "OR",
    "THE", "TO", "WAS", "WERE", "WILL", "WITH",
}

SECTOR_KEYWORDS = {
    "Technology": ["tech", "software", "technology", "cloud", "ai", "artificial intelligence",
                   "chip", "semiconductor", "digital", "platform", "app", "data", "cyber"],
    "Finance": ["bank", "financial", "finance", "investment", "trading", "market",
                "stock", "equity", "bond", "credit", "lending", "mortgage"],
    "Healthcare": ["health", "medical", "pharmaceutical", "drug", "biotech", "hospital",
                    "treatment", "patient", "fda", "clinical", "therapy"],
    "Energy": ["oil", "gas", "energy", "petroleum", "renewable", "solar", "wind",
               "electric", "power", "fuel", "drilling", "crude"],
    "Retail": ["retail", "store", "shopping", "consumer", "e-commerce", "online shopping",
               "merchandise", "sales", "retailer"],
    "Automotive": ["car", "automotive", "vehicle", "auto", "truck", "electric vehicle",
                   "ev", "manufacturing", "tesla"],
    "Real Estate": ["real estate", "property", "housing", "construction", "mortgage",
                    "development", "reit"],
    "Telecommunications": ["telecom", "communication", "wireless", "5g", "network", "internet"],
    "Aerospace": ["aerospace", "aircraft", "defense", "boeing", "space"],
    "Consumer Goods": ["consumer goods", "packaged goods", "cpg"],
}

def extract_article_text(url: str) -> str | None:
    downloaded = trafilatura.fetch_url(url)
    if not downloaded:
        return None

    text = trafilatura.extract(
        downloaded,
        include_comments=False,
        include_tables=False,
        include_formatting=False
    )
    return text

def load_ticker_map(path: Path):
    ticker_to_name = {}
    name_to_ticker = {}
    if not path.exists():
        return ticker_to_name, name_to_ticker

    with path.open("r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            ticker = (row.get("ticker") or "").strip().upper()
            name = (row.get("name") or "").strip()
            if not ticker or not name:
                continue
            ticker_to_name[ticker] = name
            name_to_ticker[name.lower()] = ticker

    return ticker_to_name, name_to_ticker


ticker_to_name, name_to_ticker = load_ticker_map(TICKER_LIST_PATH)


def fetch_articles(feed_url, max_items=30):
    feed = feedparser.parse(feed_url)
    articles = []
    for entry in feed.entries[:max_items]:
        text = extract_article_text(entry.link)
        if not text:
            continue
        articles.append({
            "title": entry.title,
            "url": entry.link,
            "published": entry.get("published"),
            "text": text,
        })
    return articles


def get_tickers(text):
    tickers = []
    for m in TICKER_RE.findall(text):
        t = m.replace("$", "").upper()
        if t in TICKER_STOP:
            continue
        if ticker_to_name and t not in ticker_to_name:
            continue
        tickers.append(t)
    return tickers


def get_companies(doc):
    companies = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    mapped = []
    for name in companies:
        ticker = name_to_ticker.get(name.lower())
        if ticker:
            mapped.append(ticker)
        else:
            mapped.append(name)
    return mapped


def get_sectors(text_lower):
    return [
        sector for sector, keywords in SECTOR_KEYWORDS.items()
        if any(kw in text_lower for kw in keywords)
    ]


def sentence_windows(sentences, idx, window=1):
    start = max(0, idx - window)
    end = min(len(sentences), idx + window + 1)
    return " ".join(sentences[start:end])


def analyze_article_entities(article, window=1):
    text = f"{article.get('title','')} {article.get('text','')}"
    if not text.strip():
        return [], [], []

    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    stock_scores = defaultdict(list)
    company_scores = defaultdict(list)
    sector_scores = defaultdict(list)

    cache = {}

    for i, sent in enumerate(sentences):
        sent_doc = nlp(sent)
        tickers = set(get_tickers(sent))
        companies = set(get_companies(sent_doc))
        sectors = set(get_sectors(sent.lower()))

        if not tickers and not companies and not sectors:
            continue

        window_text = sentence_windows(sentences, i, window=window)
        if window_text not in cache:
            cache[window_text] = get_text_sentiment_score(window_text)
        s = cache[window_text]

        for t in tickers:
            stock_scores[t].append(s)
        for c in companies:
            company_scores[c].append(s)
        for sec in sectors:
            sector_scores[sec].append(s)

    return stock_scores, company_scores, sector_scores


def aggregate_entities_with_sentiment(articles, window=1):
    stock_stats = defaultdict(lambda: {"mentions": 0, "scores": []})
    company_stats = defaultdict(lambda: {"mentions": 0, "scores": []})
    sector_stats = defaultdict(lambda: {"mentions": 0, "scores": []})

    for article in articles:
        stock_scores, company_scores, sector_scores = analyze_article_entities(article, window=window)

        for name, scores in stock_scores.items():
            stock_stats[name]["mentions"] += 1
            stock_stats[name]["scores"].append(sum(scores) / len(scores))

        for name, scores in company_scores.items():
            company_stats[name]["mentions"] += 1
            company_stats[name]["scores"].append(sum(scores) / len(scores))

        for name, scores in sector_scores.items():
            sector_stats[name]["mentions"] += 1
            sector_stats[name]["scores"].append(sum(scores) / len(scores))

    def finalize(stats):
        rows = []
        for name, data in stats.items():
            avg = sum(data["scores"]) / len(data["scores"]) if data["scores"] else 0.0
            rows.append({
                "name": name,
                "mentions": data["mentions"],
                "avg_sentiment": avg,
            })
        rows.sort(key=lambda x: x["mentions"], reverse=True)
        return rows

    return {
        "stocks": finalize(stock_stats),
        "companies": finalize(company_stats),
        "sectors": finalize(sector_stats),
    }


## Youtube Data Api Scrape

In [38]:
load_dotenv()
YOUTUBE_API_KEY = os.getenv("API_KEY")
CHANNEL_ID = "UCrp_UI8XtuYfpiqluWLD7Lw"  # CNBC channel
MAX_VIDEOS = 100

def fetch_youtube_videos_with_api(channel_id, api_key, max_results=100):
    """Fetch YouTube videos using Data API (no transcripts needed)"""
    youtube = build('youtube', 'v3', developerKey=api_key)
    videos = []
    next_page_token = None
    uploads_playlist_id = None
    
    print(f"Fetching videos from channel {channel_id}...")
    
    while len(videos) < max_results:
        try:
            # First, get the uploads playlist ID for the channel
            if uploads_playlist_id is None:  # Only need to do this once
                channel_response = youtube.channels().list(
                    part='contentDetails',
                    id=channel_id
                ).execute()
                
                if not channel_response.get('items'):
                    print(f"❌ Channel {channel_id} not found")
                    break
                
                uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
            
            # Get videos from uploads playlist
            if uploads_playlist_id:
                request = youtube.playlistItems().list(
                    part='snippet,contentDetails',
                    playlistId=uploads_playlist_id,
                    maxResults=min(max_results, max_results - len(videos)),
                    pageToken=next_page_token
                )
            else:
                # Fallback: search for videos from channel
                request = youtube.search().list(
                    part='snippet',
                    channelId=channel_id,
                    type='video',
                    maxResults=min(max_results, max_results - len(videos)),
                    pageToken=next_page_token,
                    order='date'
                )
            
            response = request.execute()
            
            # Get video IDs
            video_ids = []
            for item in response['items']:
                if 'contentDetails' in item:
                    video_ids.append(item['contentDetails']['videoId'])
                elif 'id' in item and 'videoId' in item['id']:
                    video_ids.append(item['id']['videoId'])
            
            # Get detailed video information
            if video_ids:
                video_details = youtube.videos().list(
                    part='snippet,statistics',
                    id=','.join(video_ids)
                ).execute()
                
                for item in video_details['items']:
                    snippet = item['snippet']
                    videos.append({
                        'title': snippet.get('title', ''),
                        'video_id': item['id'],
                        'url': f"https://www.youtube.com/watch?v={item['id']}",
                        'published': snippet.get('publishedAt', ''),
                        'published_date': snippet.get('publishedAt', ''),
                        'author': snippet.get('channelTitle', ''),
                        'summary': snippet.get('description', ''),  # Full description
                        'transcript_text': None,  # No transcript (IP banned)
                        'view_count': item['statistics'].get('viewCount', 0),
                        'like_count': item['statistics'].get('likeCount', 0),
                    })
            
            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break
                
            print(f"  Fetched {len(videos)} videos so far...")
            
        except Exception as e:
            print(f"❌ Error fetching videos: {e}")
            break
    
    print(f"✅ Total videos fetched: {len(videos)}")
    return videos


# Main

## Retrieve latest 100 videos

In [39]:
youtube_videos_api = fetch_youtube_videos_with_api(CHANNEL_ID, YOUTUBE_API_KEY, max_results=MAX_VIDEOS)
youtube_videos_api[:3]

Fetching videos from channel UCrp_UI8XtuYfpiqluWLD7Lw...
  Fetched 50 videos so far...
  Fetched 100 videos so far...
✅ Total videos fetched: 100


[{'title': 'Lightning Round: Sony is a buy here, says Jim Cramer',
  'video_id': 'nlAMRnmF6Ic',
  'url': 'https://www.youtube.com/watch?v=nlAMRnmF6Ic',
  'published': '2026-01-13T01:16:09Z',
  'published_date': '2026-01-13T01:16:09Z',
  'author': 'CNBC Television',
  'summary': "'Mad Money' host Jim Cramer weighs in on stocks including: CubeSmart, Sony Group, and Aquestive Therapeutics.",
  'transcript_text': None,
  'view_count': '321',
  'like_count': '13'},
 {'title': 'Mad Money 01/12/26 | Audio Only',
  'video_id': 'mNt5O1Tnd5c',
  'url': 'https://www.youtube.com/watch?v=mNt5O1Tnd5c',
  'published': '2026-01-13T01:02:04Z',
  'published_date': '2026-01-13T01:02:04Z',
  'author': 'CNBC Television',
  'summary': 'Listen to Jim Cramer’s personal guide through the confusing jungle of Wall Street investing, navigating through opportunities and pitfalls with one goal in mind - to help you make money. \n\nFor access to live and exclusive video from CNBC subscribe to CNBC PRO: https://cnb.c

## Retrieve and cache Transcripts

In [29]:
TRANSCRIPT_CACHE_PATH = Path('daily_transcripts.json')
MAX_CACHE_SIZE = 100  # Maximum number of videos to keep in cache

ytt_api = YouTubeTranscriptApi(
    proxy_config=WebshareProxyConfig(
        proxy_username = os.getenv("PROXY_USER"),
        proxy_password = os.getenv("PROXY_PASS"),
    )
)

def load_transcript_cache(path):
    if path.exists():
        try:
            content = path.read_text(encoding='utf-8')
            if content.strip():
                return json.loads(content)
            else:
                print("⚠️  Cache file is empty, starting fresh")
                return {}
        except json.JSONDecodeError as e:
            print(f"⚠️  Cache file is corrupted: {e}")
            print("   Starting with fresh cache")
            return {}
    return {}

def save_transcript_cache(path, cache):
    try:
        path.write_text(json.dumps(cache, ensure_ascii=False, indent=2), encoding='utf-8')
    except Exception as e:
        print(f"⚠️  Failed to save cache: {e}")

def fetch_transcript_with_backoff(video_id, max_retries=5):
    delay = 2.0 
    consecutive_bans = 0
    max_consecutive_bans = 10  # If 10 IPs in a row are banned, all are likely banned
    
    for attempt in range(1, max_retries + 1):
        try:
            transcript = ytt_api.fetch(video_id)
            return ' '.join([seg.text for seg in transcript])
        except (TranscriptsDisabled, NoTranscriptFound):
            return None
        except Exception as e:
            error_msg = str(e).lower()
            # Check for ban/rate limit indicators
            if any(indicator in error_msg for indicator in ['429', 'too many requests', 'rate limit', 'forbidden', '403']):
                consecutive_bans += 1
                print(f"⚠️  RATE LIMIT/BAN DETECTED (attempt {attempt}): {type(e).__name__}")
                
                if consecutive_bans >= max_consecutive_bans:
                    print(f"🚫 {max_consecutive_bans} consecutive bans - all IPs likely exhausted")
                    raise Exception("ALL_IPS_BANNED")
                
                # Wait and let Webshare rotate to next IP
                wait_time = 3 + random.random() * 2
                print(f"   Waiting {wait_time:.1f}s for IP rotation, then retrying...")
                time.sleep(wait_time)
                continue  # Retry with next IP from rotation
            
            print(f"Attempt {attempt} failed for {video_id}: {type(e).__name__}: {str(e)[:100]}")
            if attempt == max_retries:
                return None
            time.sleep(delay + random.random() * 2)
            delay *= 2
    
    return None

def attach_transcripts(videos, cache_path=TRANSCRIPT_CACHE_PATH, max_cache_size=MAX_CACHE_SIZE):
    latest_ids = [v.get('video_id') for v in videos if v.get('video_id')]
    total_videos = len(latest_ids)
    print(f"\n📝 Processing {total_videos} videos for transcripts...\n")
    print(f"🌐 Using Webshare rotating proxy (10 IPs in pool)\n")

    # Load full cache
    cache = load_transcript_cache(cache_path)
    old_cache_size = len(cache)
    
    # Create ordered list: newest videos first
    # Assuming videos list is already ordered from newest to oldest
    all_video_ids = latest_ids.copy()
    
    # Add old cached videos that aren't in the new list (to maintain order)
    for old_vid in cache.keys():
        if old_vid not in all_video_ids:
            all_video_ids.append(old_vid)
    
    # Keep only the newest MAX_CACHE_SIZE videos
    videos_to_keep = all_video_ids[:max_cache_size]
    
    # Filter cache to only keep videos we want
    filtered_cache = {vid: cache[vid] for vid in videos_to_keep if vid in cache}
    removed_count = old_cache_size - len(filtered_cache)
    
    print(f"📦 Cache status: {old_cache_size} total → keeping {len(filtered_cache)} (removed {removed_count} oldest)\n")
    
    cache = filtered_cache

    success_count = 0
    failed_count = 0
    cached_count = 0
    all_ips_banned = False

    for idx, video in enumerate(videos, start=1):
        vid = video.get('video_id')
        if not vid:
            continue
        
        if vid in cache:
            video['transcript_text'] = cache[vid]
            cached_count += 1
            print(f"[{idx}/{total_videos}] ✓ Cached: {vid} - {video.get('title', 'N/A')[:50]}")
            if cache[vid]:
                print(f"  Preview: {cache[vid][:150]}...\n")
            continue
        
        time.sleep(1.5 + random.random() * 1.5)
        
        try:
            transcript_text = fetch_transcript_with_backoff(vid)
        except Exception as e:
            if "ALL_IPS_BANNED" in str(e):
                all_ips_banned = True
                print(f"\n🚫 ALL IPs BANNED - Stopping transcript fetching and saving cache...")
                video['transcript_text'] = None
                cache[vid] = None
                save_transcript_cache(cache_path, cache)
                break
            transcript_text = None
        
        video['transcript_text'] = transcript_text
        cache[vid] = transcript_text
        
        if transcript_text:
            success_count += 1
            print(f"[{idx}/{total_videos}] ✓ Fetched: {vid} - {video.get('title', 'N/A')[:50]}")
            print(f"  Preview: {transcript_text[:150]}...\n")
        else:
            failed_count += 1
            print(f"[{idx}/{total_videos}] ✗ Failed: {vid} - {video.get('title', 'N/A')[:50]}\n")
        
        if idx % 10 == 0:
            save_transcript_cache(cache_path, cache)
            print(f"  💾 Cache saved at {idx} videos\n")

    # Final save
    save_transcript_cache(cache_path, cache)
    
    print(f"\n📊 Summary:")
    print(f"  ✓ Successfully fetched: {success_count}")
    print(f"  ✓ From cache: {cached_count}")
    print(f"  ✗ Failed/No transcript: {failed_count}")
    print(f"  Total processed: {idx}/{total_videos}")
    print(f"  📦 Final cache size: {len(cache)}/{max_cache_size}")
    if all_ips_banned:
        print(f"  🚫 Stopped early - all rotating IPs exhausted")
    print()
    
    return videos

youtube_videos_api = attach_transcripts(youtube_videos_api)


📝 Processing 100 videos for transcripts...

🌐 Using Webshare rotating proxy (10 IPs in pool)

📦 Cache status: 100 total → keeping 14 (removed 86 oldest)

[1/100] ✓ Fetched: nlAMRnmF6Ic - Lightning Round: Sony is a buy here, says Jim Cram
  Preview: STOCK PRESENTATION. MY STAFF PREPARES THE GRAPHICS. WE PLAN THIS OUT AND THEN THE LIGHTNING ROUND IS OVER. ARE YOU READY, SKI! DADDY! LANCE AND MERLIN...

[2/100] ✓ Fetched: mNt5O1Tnd5c - Mad Money 01/12/26 | Audio Only
  Preview: Hey, I'm Kramer. Welcome to a special Mad Money from One Market in San Francisco. Welcome to Cray America. Other people, my friends, I'm just trying t...

[3/100] ✓ Fetched: JrpOoIJf1tY - Novo Nordisk CEO: Wegovy not only reduces your wei
  Preview: BEHIND OZEMPIC AND WEGOVY, JUST LAUNCHED A PILL VERSION OF THE REVOLUTIONARY WEIGHT LOSS AND DIABETES DRUG. SO WHILE WE'RE OUT HERE IN SAN FRANCISCO F...

[4/100] ✓ Fetched: RFRKcH7sNuc - Novo Nordisk CEO: People have been waiting for a G
  Preview: AROUND TEN, 15 MILL

## Add transcripts to dict

In [ ]:
# Overwrite in-memory transcripts with cache values when available
import json
from pathlib import Path

def refresh_transcripts_from_cache(videos, cache_path=Path('daily_transcripts.json')):
    if not cache_path.exists():
        return videos
    cache = json.loads(cache_path.read_text(encoding='utf-8'))
    updated = 0
    for video in videos:
        vid = video.get('video_id')
        if not vid:
            continue
        cached_value = cache.get(vid)
        if cached_value is not None:
            if video.get('transcript_text') != cached_value:
                video['transcript_text'] = cached_value
                updated += 1
    print(f'Overwrote {updated} transcripts from dict')
    return videos

youtube_videos_api = refresh_transcripts_from_cache(youtube_videos_api)


Overwrote 97 transcripts from cache


## Check rotating proxies

In [65]:
# username = os.getenv("PROXY_USER")
# password = os.getenv("PROXY_PASS")
# endpoint = os.getenv("PROXY_HOST") +  ":" + str(os.getenv("PROXY_PORT"))

# proxy_url = f"http://{username}:{password}@{endpoint}"

# def check_ip_rotation(num_requests=10):
#     """Check if proxy IPs are rotating"""
#     proxies = {
#         'http': proxy_url,
#         'https': proxy_url
#     }
    
#     ips = []
#     for i in range(num_requests):
#         try:
#             # Using http instead of https for simpler testing
#             response = requests.get('http://ipinfo.io/json', 
#                                    proxies=proxies, 
#                                    timeout=10)
#             ip = response.json().get('ip')
#             ips.append(ip)
#             print(f"Request {i+1}: IP = {ip}")
#         except Exception as e:
#             print(f"Request {i+1} failed: {e}")
    
#     unique_ips = set(ips)
#     print(f"\nTotal requests: {len(ips)}")
#     print(f"Unique IPs: {len(unique_ips)}")
#     print(f"IPs are {'ROTATING ✓' if len(unique_ips) > 1 else 'NOT ROTATING ✗'}")
#     return ips

# # Test rotation
# print("Testing IP rotation with Webshare:")
# check_ip_rotation(10)

# After Transcript

## Sentiment and Summarizer

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import logging
logging.set_verbosity_error()

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sentiment_analyzer = pipeline("sentiment-analysis", model="ProsusAI/finbert")

def chunk_text_words(text, chunk_words=500):
    words = text.split()
    return [
        " ".join(words[i:i+chunk_words])
        for i in range(0, len(words), chunk_words)
    ]

def summarize_long_text(text):
    chunks = chunk_text_words(text, chunk_words=500)

    partial_summaries = []
    for chunk in chunks:
        result = summarizer(
            chunk,
            max_length=120,
            min_length=40,
            do_sample=False,
            truncation=True
        )
        partial_summaries.append(result[0]["summary_text"])

    combined = " ".join(partial_summaries)

    final = summarizer(
        combined,
        max_length=180,
        min_length=60,
        do_sample=False,
        truncation=True
    )

    return final[0]["summary_text"]


def analyze_video_sentiment(video, debug=False):
    """Analyze sentiment of title and transcript separately"""
    
    # Title: Direct sentiment (no summarization)
    title = video.get('title', '')
    if title:
        try:
            title_sentiment = sentiment_analyzer(title[:512])[0]
            video['title_sentiment'] = title_sentiment
        except Exception as e:
            if debug:
                print(f"    Title sentiment failed: {e}")
            video['title_sentiment'] = None
    else:
        video['title_sentiment'] = None
    
    # Transcript: Summarize → Sentiment
    transcript_text = video.get('transcript_text', '')
    
    if debug:
        print(f"   Transcript length: {len(transcript_text)} chars, {len(transcript_text.split())} words")
    
    # Check if transcript exists and is long enough
    if not transcript_text or len(transcript_text.strip()) < 200:
        if debug:
            print(f"    Transcript too short or missing")
        video['transcript_summary'] = None
        video['transcript_sentiment'] = None
        return video
    
    try:
        # Clean and truncate transcript
        transcript_text = transcript_text.strip()
        words = transcript_text.split()
        
        if debug:
            print(f"   Word count: {len(words)}")
        
        # BART works best with 100-1024 tokens
        if len(words) < 100:
            if debug:
                print(f"    Too few words: {len(words)}")
            video['transcript_summary'] = None
            video['transcript_sentiment'] = None
            return video
        
        if len(words) > 1000:
            transcript_text = ' '.join(words[:1000])
            if debug:
                print(f"    Truncated to 1000 words")
        
        if debug:
            print(f"   Generating summary...")
            print(f"   First 200 chars: {transcript_text[:200]}")
        
        # Generate summary with better parameters

        summary = summarize_long_text(transcript_text)
        
        if debug:
            print(f"   Summary: {summary}")
        
        # Sentiment of summary
        transcript_sentiment = sentiment_analyzer(summary[:512])[0]
        
        if debug:
            print(f"   Sentiment: {transcript_sentiment}")
        
        video['transcript_summary'] = summary
        video['transcript_sentiment'] = transcript_sentiment
        
    except Exception as e:
        if debug:
            print(f"   Error: {type(e).__name__}: {str(e)}")
        video['transcript_summary'] = None
        try:
            video['transcript_sentiment'] = sentiment_analyzer(transcript_text[:512])[0]
        except Exception:
            video['transcript_sentiment'] = None
    
    return video

# Test on first video with debug output
print("\nTesting first video with debug output:\n")
if youtube_videos_api:
    test_video = youtube_videos_api[0].copy()
    print(f"Title: {test_video.get('title')}")
    analyze_video_sentiment(test_video, debug=True)
    print("\n" + "="*80 + "\n")

# Ask user if they want to continue
response = input("Continue with all videos? (y/n): ")

if response.lower() == 'y':
    print("\n Analyzing all videos...")
    
    for video in tqdm(youtube_videos_api, desc="Processing videos", unit="video"):
        if video.get('transcript_text'):
            analyze_video_sentiment(video, debug=False)
        else:
            video['title_sentiment'] = None
            video['transcript_summary'] = None
            video['transcript_sentiment'] = None
    
    print("Analysis complete!\n")


Testing first video with debug output:

Title: Lightning Round: Sony is a buy here, says Jim Cramer
   Transcript length: 1632 chars, 316 words
   Word count: 316
   Generating summary...
   First 200 chars: STOCK PRESENTATION. MY STAFF PREPARES THE GRAPHICS. WE PLAN THIS OUT AND THEN THE LIGHTNING ROUND IS OVER. ARE YOU READY, SKI! DADDY! LANCE AND MERLIN. LANCE. >> THANK YOU FOR TAKING MY CALL. MR. CRAM
   Summary: Jim's stock is down a dollar and a half from his recent high. A company CEO says the stock is undervaluing the company. A five-year-old boy thanks Jim for making his money. The stock market is down 1.7% this year. The Dow Jones Industrial Average is down 0.3%. The S&P 500 is up 0.2%.
   Sentiment: {'label': 'negative', 'score': 0.9727199077606201}



 Analyzing all videos...


Processing videos: 100%|██████████| 100/100 [20:54<00:00, 12.54s/video]

Analysis complete!



In [ ]:
def save_to_json(videos, filename):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(videos, f, ensure_ascii=False, indent=2)

# usage
save_to_json(youtube_videos_api, "youtube_analysis.json")

## Aggregation

In [62]:
def sentiment_to_score(sentiment):
    if not sentiment:
        return None
    label = str(sentiment.get('label', '')).upper()
    score = float(sentiment.get('score', 0))
    if 'POS' in label:
        return score
    if 'NEG' in label:
        return -score
    return 0.0

def extract_video_text(video, prefer_summary=True):
    title = video.get('title', '')
    transcript = ''
    if prefer_summary and video.get('transcript_summary'):
        transcript = video['transcript_summary']
    elif video.get('transcript_text'):
        transcript = video['transcript_text']
    combined = f"{title} {transcript}".strip()
    return combined

def analyze_video_entities_split(video):
    title = video.get('title', '') or ''
    summary = video.get('transcript_summary', '') or ''

    title_doc = nlp(title) if title else None
    summary_doc = nlp(summary) if summary else None

    title_tickers = set(get_tickers(title)) if title else set()
    title_companies = set(get_companies(title_doc)) if title_doc else set()
    title_sectors = set(get_sectors(title.lower())) if title else set()

    summary_tickers = set(get_tickers(summary)) if summary else set()
    summary_companies = set(get_companies(summary_doc)) if summary_doc else set()
    summary_sectors = set(get_sectors(summary.lower())) if summary else set()

    title_score = sentiment_to_score(video.get('title_sentiment'))
    summary_score = sentiment_to_score(video.get('transcript_sentiment'))

    return {
        "title": (title_tickers, title_companies, title_sectors, title_score),
        "summary": (summary_tickers, summary_companies, summary_sectors, summary_score),
    }

def aggregate_youtube_entities(videos):

    def new_bucket():
        return {
            "title_mentions": 0,
            "title_scores": [],
            "summary_mentions": 0,
            "summary_scores": [],
        }

    stock_stats = defaultdict(new_bucket)
    company_stats = defaultdict(new_bucket)
    sector_stats = defaultdict(new_bucket)

    for video in videos:
        parts = analyze_video_entities_split(video)

        for part_name, (tickers, companies, sectors, score) in parts.items():

            for t in tickers:
                stock_stats[t][f"{part_name}_mentions"] += 1
                if score is not None:
                    stock_stats[t][f"{part_name}_scores"].append(score)

            for c in companies:
                company_stats[c][f"{part_name}_mentions"] += 1
                if score is not None:
                    company_stats[c][f"{part_name}_scores"].append(score)

            for s in sectors:
                sector_stats[s][f"{part_name}_mentions"] += 1
                if score is not None:
                    sector_stats[s][f"{part_name}_scores"].append(score)

    def finalize(stats):
        rows = []
        for name, data in stats.items():
            rows.append({
                "name": name,

                "title_mentions": data["title_mentions"],
                "avg_title_sentiment": (
                    sum(data["title_scores"]) / len(data["title_scores"])
                    if data["title_scores"] else None
                ),

                "summary_mentions": data["summary_mentions"],
                "avg_summary_sentiment": (
                    sum(data["summary_scores"]) / len(data["summary_scores"])
                    if data["summary_scores"] else None
                ),
            })

        rows.sort(key=lambda x: (x["title_mentions"] + x["summary_mentions"]), reverse=True)
        return rows

    return {
        "stocks": finalize(stock_stats),
        "companies": finalize(company_stats),
        "sectors": finalize(sector_stats),
    }

In [63]:
result = aggregate_youtube_entities(youtube_videos_api)

In [64]:
result

{'stocks': [{'name': 'META',
   'title_mentions': 0,
   'avg_title_sentiment': None,
   'summary_mentions': 3,
   'avg_summary_sentiment': 0.35367051760355633},
  {'name': 'ED',
   'title_mentions': 0,
   'avg_title_sentiment': None,
   'summary_mentions': 2,
   'avg_summary_sentiment': -0.4860196113586426},
  {'name': 'ALL',
   'title_mentions': 0,
   'avg_title_sentiment': None,
   'summary_mentions': 2,
   'avg_summary_sentiment': -0.4860196113586426},
  {'name': 'DG',
   'title_mentions': 1,
   'avg_title_sentiment': 0.0,
   'summary_mentions': 0,
   'avg_summary_sentiment': None},
  {'name': 'GOOG',
   'title_mentions': 0,
   'avg_title_sentiment': None,
   'summary_mentions': 1,
   'avg_summary_sentiment': 0.0},
  {'name': 'COR',
   'title_mentions': 0,
   'avg_title_sentiment': None,
   'summary_mentions': 1,
   'avg_summary_sentiment': -0.4934401512145996},
  {'name': 'LOW',
   'title_mentions': 0,
   'avg_title_sentiment': None,
   'summary_mentions': 1,
   'avg_summary_sentim

In [ ]:
save_to_json(result, "entity_mentions.json")